In [1]:
import torch
import torch.nn as nn
from torchvision import datasets,transforms
import torchvision.models as tvmodel
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
import cv2
import numpy as np

In [2]:
from DTW import *
from image_pro import *

In [3]:
class YOLOv1_2_resnet(nn.Module):
    def __init__(self):
        super(YOLOv1_2_resnet,self).__init__()
        #resnet = tvmodel.resnet34(pretrained=True)  
        resnet = tvmodel.resnet18(pretrained=True)
        resnet_out_channel = resnet.fc.in_features  # 记录resnet全连接层之前的网络输出通道数，方便连入后续卷积网络中
        self.resnet = nn.Sequential(*list(resnet.children())[:-2])  # 去除resnet的最后两层
        # 以下是YOLOv1的最后四个卷积层
        self.Conv_layers_1 = nn.Sequential(
            nn.Conv2d(resnet_out_channel,256,3,padding=1),
            nn.BatchNorm2d(256),  # 为了加快训练，这里增加了BN层，原论文里YOLOv1是没有的
            nn.LeakyReLU(),
            nn.Conv2d(256,256,3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(256,128,3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(128,48,1,stride=1,padding=0),
            nn.BatchNorm2d(48),  #14*14
        )
        self.Conv_layers_2 = nn.Sequential(
            nn.LeakyReLU(),
            nn.Conv2d(48,128,3,stride=2,padding=1),   #7*7
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
        )
        # 以下是YOLOv1的最后2个全连接层
        self.Conn_layers = nn.Sequential(
            nn.Linear(7*7*128,2048),
            nn.LeakyReLU(),
            nn.Linear(2048,7*7*30),
            nn.Sigmoid()  
        )

    def forward(self, input):                #训练时的前向传播函数
        input = self.resnet(input)
        input = self.Conv_layers_1(input)
        input = self.Conv_layers_2(input)
        input = input.view(input.size()[0],-1)
        input = self.Conn_layers(input)
        return input.reshape(-1, (5*NUM_BBOX+len(CLASSES)), 7, 7)  
    def forward_2(self, input):               #提取特征时的前向传播函数
        input = self.resnet(input)
        input = self.Conv_layers_1(input)
        return input

In [4]:
model = torch.load("./models2_pkl/YOLOv1_2_epoch30.pkl")
model = model.cpu()

In [5]:
part1 = get_image_input('A1.jpg',4)
part1 = img2tensor(part1)
part2 = get_image_input('A2.jpg',5)
part2 = img2tensor(part2)
y1 = model.forward_2(part1)
y2 = model.forward_2(part2)
constant = DTW_2D(y1,y2).item()
print(constant)

2158.984130859375


In [6]:
IMAGE_PATH = "./test_images/"
files = os.listdir(IMAGE_PATH)
initimg = get_image_input('A1.jpg',3)
initimg = img2tensor(initimg)
y1 = model.forward_2(initimg)
for filename in files:
    if filename == 'A1.jpg':
        continue
    testimg = get_image_input(filename,3)
    testimg = img2tensor(testimg)
    y2 = model.forward_2(testimg)
    res = DTW_2D(y1,y2).item()
    print('A1.jpg和'+filename+'的距离为：'+str(res-constant))

A1.jpg和A2.jpg的距离为：25.5859375
A1.jpg和B.jpg的距离为：405.435791015625
A1.jpg和C.jpg的距离为：532.007080078125
A1.jpg和D.jpg的距离为：239.071044921875
A1.jpg和E.jpg的距离为：698.514404296875
A1.jpg和F.jpg的距离为：498.42431640625
A1.jpg和G.jpg的距离为：917.935302734375
A1.jpg和H.jpg的距离为：561.505859375


In [6]:
part1 = get_image_input('01.png',4)
part1 = img2tensor(part1)
part2 = get_image_input('01.png',5)
part2 = img2tensor(part2)
y1 = model.forward_2(part1)
y2 = model.forward_2(part2)
constant = DTW_2D(y1,y2).item()
print(constant)

initimg = get_image_input('01.png',2)
initimg = img2tensor(initimg)
y1 = model.forward_2(initimg)

testimg = get_image_input('02.png',2)
testimg = img2tensor(testimg)
y2 = model.forward_2(testimg)
res = DTW_2D(y1,y2).item()
print('01.png和02.png的距离为：'+str(res-constant))

testimg = get_image_input('03.png',2)
testimg = img2tensor(testimg)
y2 = model.forward_2(testimg)
res = DTW_2D(y1,y2).item()
print('01.jpg和03.png的距离为：'+str(res-constant))

1077.477783203125
01.png和02.png的距离为：224.7496337890625
01.jpg和03.png的距离为：387.0625
